In [1]:
from yoloDetector import YoloDetector

In [2]:
!pip install numba

## Khoi tao: model, label_path va image_path

In [3]:
detector = YoloDetector() #initialize the detector
label_path = r"C:/data_object_image_2/training/label_2"
image_path = r"C:/data_object_image_2/training/image_2"

In [4]:
import os
import numpy as np
import cv2

In [5]:
print(len(os.listdir(label_path)))

7481


In [6]:
print(len(os.listdir(image_path)))

7481


### Dem so type


In [7]:
labels = set()
for one_path in os.listdir(label_path):
    with open(os.path.join(label_path, one_path), 'r') as f:
            
                for line in f:
                    #preprocess path
                    line = line.strip().split(' ')
                    label = line[0]
                    if label not in labels:
                        labels.add(label)
print(labels)

## Bat dau chay tu doan code nay

In [ ]:
def IoU(box1, box2):
   
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    iou = 0.0

    
    xA = max(x1, x2)
    yA = max(y1, y2)
    xB = min(x1 + w1, x2 + w2)
    yB = min(y1 + h1, y2 + h2)

    interArea = max(0, xB - xA) * max(0, yB - yA)

    box1Area = w1 * h1
    box2Area = w2 * h2

    iou = interArea / float(box1Area + box2Area - interArea)

    return iou

In [ ]:
import tqdm

def evaluate_detector(min_pixel, max_occlusion, max_truncation, detector, path = 'data/test_ann.txt', images_path = '',):
    TP = [0, 0, 0, 0, 0]
    FP = [0, 0, 0, 0, 0]
    FN = [0, 0, 0, 0, 0]

    ### tạo đối tượng CascadeClassifier và khởi tạo bằng file xml đã học được 
    # our_cascade = cv2.CascadeClassifier('cascade.xml')

    threshold = 0.3
    cnt = 0
    
    for one_path in tqdm.tqdm(os.listdir(path), desc="Processing", unit="file"):
    # for one_path in ['000003.txt']:
        # print(one_path)
        img = cv2.imread(os.path.join(images_path, one_path[:-3] + 'png'))
        # print(img)
        detector.DetectFrame(img)
        cars = []
        pedestrian = []
        truck = []
        bicycle = []
        other = []
        for i in detector._object_info:
            if i.label == 'car':
                cars.append(i)
            elif i.label == 'person':
                pedestrian.append(i)
            elif i.label == 'truck':
                truck.append(i)
            elif i.label == 'bicycle':
                bicycle.append(i)
            else:
                other.append(i)
        true_detect = [0,0,0,0,0]
        with open(os.path.join(path, one_path), 'r') as f:
        
            for line in f:
                #preprocess path
                line = line.strip().split(' ') #
                
                truth_box = [float(line[i]) for i in range(4, 8)]   # x1, y1, x2, y2
                truth_box[2] = truth_box[2] - truth_box[0]
                truth_box[3] = truth_box[3] - truth_box[1]
                # if(truth_box[3] < min_pixel): print('skip: pixel'); continue
                # if(float(line[1]) > max_truncation): print('skip: max_truncation');continue
                # if(float(line[2]) > max_occlusion): print('skip: max_occulusion'); continue
                type = line[0]
                if type == 'Car':
                    if len(cars) == 0:
                        FN[0] += 1
                        continue
                    for detection in cars:
                        box = (detection.x, detection.y, detection.width, detection.height)
                        
                        t = IoU(box, truth_box)
                        if t >= threshold:
                            true_detect[0] += 1
                            break
                    else:
                        FN[0] += 1
                elif type == 'Pedestrian':
                    if len(pedestrian) == 0:
                        FN[1] += 1
                        continue
                    for detection in pedestrian:
                        box = (detection.x, detection.y, detection.width, detection.height)
                        t = IoU(box, truth_box)
                        if t >= threshold:
                            true_detect[1] += 1
                            break
                    else:
                        FN[1] += 1
                elif type == 'Truck':
                    if len(truck) == 0:
                        FN[2] += 1
                        continue
                    for detection in truck:
                        box = (detection.x, detection.y, detection.width, detection.height)
                        t = IoU(box, truth_box)
                        if t >= threshold:
                            true_detect[2] += 1
                            break
                    else:
                        FN[2] += 1
                elif type == 'Cyclist':
                    if len(bicycle) == 0:
                        FN[3] += 1
                        continue
                    for detection in bicycle:
                        box = (detection.x, detection.y, detection.width, detection.height)
                        t = IoU(box, truth_box)
                        if t >= threshold:
                            true_detect[3] += 1
                            break
                    else:
                        FN[3] += 1
                else:
                    if len(other) == 0:
                        FN[4] += 1
                        continue
                    for detection in other:
                        box = (detection.x, detection.y, detection.width, detection.height)
                        t = IoU(box, truth_box)
                        if t >= threshold:
                            true_detect[4] += 1
                            break
                    else:
                        FN[4] += 1

        for i in range(5):
            TP[i] += true_detect[i]
            if i == 0: FP[i] += len(cars) - true_detect[i]
            elif i == 1: FP[i] += len(pedestrian) - true_detect[i]
            elif i == 2: FP[i] += len(truck) - true_detect[i]
            elif i == 3: FP[i] += len(bicycle) - true_detect[i]
            elif i == 4: FP[i] += len(other) - true_detect[i]

        cnt += 1
        
    TP = np.array(TP)
    FP = np.array(FP)
    FN = np.array(FN)
    precision = 1.0 * TP / (TP + FP)
    recall = 1.0 * TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    
    return precision, recall, f1

### Easy task: height of pixel >= 40, truncated <= 0.15, ko bi che phu (0)

In [49]:
#easy task
precision, recall, f1 = evaluate_detector(40,0,0.15, detector, label_path, images_path = image_path)

6.683598449405159 %


In [48]:
print('precision: ', precision)
# print(precision, recall, f1, sep = '\n')

[0.90937997 0.72093023 0.375      0.25       0.1875    ]
[0.77823129 0.59047619 0.5        0.15217391 0.01627486]
[0.83870968 0.64921466 0.42857143 0.18918919 0.02995008]


In [ ]:
import pandas as pd
object_types = ["Car", "Pedestrian", "Truck", "Cyclist", "Other"]

# Tạo DataFrame từ các giá trị
df = pd.DataFrame({
    "Object Type": object_types,
    "Precision": precision,
    "Recall": recall,
    "F1": f1
})

# Hiển thị DataFrame dưới dạng bảng
print(df.to_string(index=False))

## Medium task: height of pixel < 25,>

In [21]:
#Medium task
precision, recall, f1 = evaluate_detector(25,1,0.3, detector, label_path, images_path = image_path)

Processing: 100%|██████████| 7481/7481 [33:09<00:00,  3.76file/s]  


In [22]:
import pandas as pd
object_types = ["Car", "Pedestrian", "Truck", "Cyclist", "Other"]

# Tạo DataFrame từ các giá trị
df = pd.DataFrame({
    "Object Type": object_types,
    "Precision": precision,
    "Recall": recall,
    "F1": f1
})

# Hiển thị DataFrame dưới dạng bảng
print(df.to_string(index=False))

Object Type  Precision   Recall       F1
        Car   0.899544 0.803180 0.848635
 Pedestrian   0.770043 0.633608 0.695195
      Truck   0.323064 0.354662 0.338126
    Cyclist   0.108491 0.141364 0.122765
      Other   0.219926 0.033428 0.058034


In [ ]:
print(precision, recall, f1, sep = '\n')

# Hard task

In [ ]:
#hard task
precision, recall, f1 = evaluate_detector(25,2,0.5, detector, label_path, images_path = image_path)

Processing:   0%|          | 0/7481 [00:00<?, ?file/s]

Processing:  26%|██▋       | 1972/7481 [13:14<36:26,  2.52file/s]  

In [ ]:
import pandas as pd
object_types = ["Car", "Pedestrian", "Truck", "Cyclist", "Other"]

# Tạo DataFrame từ các giá trị
df = pd.DataFrame({
    "Object Type": object_types,
    "Precision": precision,
    "Recall": recall,
    "F1": f1
})

# Hiển thị DataFrame dưới dạng bảng
print(df.to_string(index=False))